# FLAN-T5

## What is [FLAN-T5](https://arxiv.org/pdf/2210.11416.pdf)?

* FLAN-T5 is an open-source, sequence-to-sequence language model developed by Google researchers in late 2022. It is capable of performing various natural language processing tasks and can be used both in research and commercial applications. The model is based on the Transformer architecture and trained on a large corpus of text known as the Colossal Clean Crawled Corpus (C4).

* Fine-tuning FLAN-T5 is essential to adapt it to specific tasks and improve its performance. This process allows customization of the model according to the user's needs and data, making it accessible to a wider range of users, including smaller organizations and individual researchers without GPU resources.

* Instruction fine-tuning is a technique used to customize pre-trained large language models (LLMs) to perform specific tasks based on explicit instructions or demonstrations. Unlike traditional fine-tuning, which involves training a model on task-specific data, instruction fine-tuning provides higher-level guidance to the model to shape its behavior according to predefined instructions.

  * Instruction: Analyze the sentiment of the text and determine whether it's positive or negative.

  * Input_text: "I thoroughly enjoyed the movie; it was fantastic."

  * Output_text: Positive, Negative

## Potential Applications
Potential applications of fine-tuned FLAN-T5 include:
- **Text summarization:** Condensing large amounts of text into concise summaries..
- **Text Classification:** Categorizing text data into predefined classes (e.g., spam or non-spam, positive or negative sentiment)..
- **Domain-Specific Question Answering:** Whether it's medical, legal, or technical domains, fine-tuned FLAN-T5 models can be tailored to answer questions specific to different industries or subjects with high accuracy.

Fine-tuning FLAN-T5 opens up possibilities for optimizing its performance in various real-world scenarios.


### Library

In [ ]:
!pip install nltk
!pip install datasets
!pip install transformers[torch]
!pip install tokenizers
!pip install evaluate
!pip install rouge_score
!pip install sentencepiece
!pip install huggingface_hub
!pip install tabulate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86

### Imports

In [ ]:
import nltk
import evaluate
import pandas as pd
import numpy as np
from tabulate import tabulate
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

### Dataset preparation

In [ ]:
import pandas as pd

# Given dataset
data = pd.read_csv('/content/features.csv')

# Create DataFrame
df = pd.DataFrame(data)

# Function to generate input and output
def generate_input_output(row):
    # Formulate the question
    input_text = (f"For a {row['Gender']} customer with an annual income of ${row['Annual_Income']}, who bought a {row['Color']} {row['Company']} {row['Model']} "
                  f"({row['Engine']} engine) from {row['Dealer_Name']} in {row['City']} on {row['Year']}-{row['Month']}-{row['Day']}, priced at ${row['Price_($)']}, "
                  f"what was the claim amount?")

    # Provide the answer
    output_text = row['Claim_amount']

    return input_text, output_text

# Generate input_output pairs using list comprehension
input_output_pairs = [generate_input_output(row) for _, row in df.iterrows()]

# Extract input and output into separate lists
input_text, output_text = zip(*input_output_pairs)

# Create a DataFrame from the lists
question_answer_df = pd.DataFrame({'input_text': input_text, 'output_text': output_text})

# Save the dataframe to a file
question_answer_df.to_csv('/content/datasetqa.csv', index=False)


In [ ]:
def print_df(df):
    print(tabulate(df.head(5), headers=df.columns, tablefmt='psql'))

print_df(question_answer_df)

+----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|    | input_text                                                                                                                                                                                                                                |   output_text |
|----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------|
|  0 | For a Male customer with an annual income of $13500, who bought a Black Ford Expedition (DoubleÃ‚Â Overhead Camshaft engine) from Buddy Storbeck's Diesel Service Inc in Riga on 2022-Jan-2, priced at $26000, what was 

In [ ]:
import pandas as pd
import json

def format_and_save_to_json(input_csv_path, output_json_path):
    # Read data from CSV
    data = pd.read_csv(input_csv_path)

    # Convert DataFrame to list of dictionaries in desired format
    formatted_data = []
    for idx, row in data.iterrows():
        formatted_data.append({
            "input_text": row['input_text'],
            "output_text": row['output_text'],
            "id": str(idx)  # Adding an ID based on index
        })

    # Save data as JSON
    with open(output_json_path, "w") as json_file:
        json.dump(formatted_data, json_file, indent=4)

# Example usage
input_csv_path = "/content/datasetqa.csv"
output_json_path = "/content/formatted.json"

format_and_save_to_json(input_csv_path, output_json_path)


In [ ]:
import json

def print_json_rows(json_file_path, num_rows):
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)

    # Print specified number of rows
    for i in range(num_rows):
        print(json.dumps(data[i], indent=4))

# Specify the number of rows you want to print
num_rows_to_print = 5

# Call the function to print rows from the JSON file
print_json_rows(output_json_path, num_rows_to_print)

{
    "input_text": "For a Male customer with an annual income of $13500, who bought a Black Ford Expedition (Double\u00c3\u201a\u00c2\u00a0Overhead Camshaft engine) from Buddy Storbeck's Diesel Service Inc in Riga on 2022-Jan-2, priced at $26000, what was the claim amount?",
    "output_text": 0.0,
    "id": "0"
}
{
    "input_text": "For a Male customer with an annual income of $1480000, who bought a Black Dodge Durango (Double\u00c3\u201a\u00c2\u00a0Overhead Camshaft engine) from C & M Motors Inc in Liepaja on 2022-Jan-2, priced at $19000, what was the claim amount?",
    "output_text": 1900.0,
    "id": "1"
}
{
    "input_text": "For a Male customer with an annual income of $1035000, who bought a Red Cadillac Eldorado (Overhead Camshaft engine) from Capitol KIA in Riga on 2022-Jan-2, priced at $31500, what was the claim amount?",
    "output_text": 0.0,
    "id": "2"
}
{
    "input_text": "For a Male customer with an annual income of $13500, who bought a Pale White Toyota Celica (O

#Loading the model

## Google's FLAN-T5 Models on Hugging Face

Here are five publicly available versions of Google's FLAN-T5 models hosted on Hugging Face:


## [flan-t5-base](https://huggingface.co/google/flan-t5-base):

- **Parameters:** 250 Million
- **Download size:** 1 GB
- **Best suited for:** Balancing performance and resource requirements for various tasks, including text classification, summarization, and code generation.

In [ ]:
# Load the tokenizer, model, and data collator

MODEL_NAME = "google/flan-t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

### HuggingFace Datasets

HuggingFace Datasets is a lightweight library designed for sharing and accessing datasets and evaluation metrics for Natural Language Processing (NLP). It offers compatibility with NumPy, Pandas, PyTorch, and TensorFlow, and features a user-friendly interface. With built-in interoperability and efficient memory usage, it provides access to a rich collection of NLP datasets and evaluation metrics, making it easy for the community to add and share new datasets. Originating from TensorFlow Datasets, it offers smart caching mechanisms and is optimized for handling large datasets effectively.

In [ ]:
from datasets import load_dataset

# this dataset uses the new Image feature :)
question_answer_da = load_dataset('/content/dataset.py')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:929: FutureWarning: The repository for dataset contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at /content/dataset.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
question_answer_da = question_answer_da["train"].train_test_split(test_size=0.2)
# Check the length of the data and its structure
question_answer_da

DatasetDict({
    train: Dataset({
        features: ['id', 'input_text', 'output_text'],
        num_rows: 19124
    })
    test: Dataset({
        features: ['id', 'input_text', 'output_text'],
        num_rows: 4782
    })
})

In [ ]:
# instruction of our tasks with "output_text the input_text"
instruction = "Please answer this question: This is a Insurance car claim data for time series . Where you need to predict if a customer make a insurance claim or not. Your task is to predict the accurately forecast the claim amount based on the input  "

# Define the preprocessing function

def preprocess_function(examples):
   """Add instruction to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [instruction + doc for doc in examples["input_text"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)

   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=examples["output_text"],
                      max_length=512,
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

# Map the preprocessing function across our dataset
tokenized_dataset = question_answer_da.map(preprocess_function, batched=True)

Map:   0%|          | 0/19124 [00:00<?, ? examples/s]

Map:   0%|          | 0/4782 [00:00<?, ? examples/s]

Two commonly used metrics in the field of NLP evaluation are BLEU and ROUGE scores.

#### BLEU Score:

- BLEU is primarily used in machine translation tasks, where the goal is to assess the quality of machine-generated translations compared to human-generated reference translations.
- It measures the precision of n-grams (contiguous sequences of n words) in the machine-generated translation by comparing them to the reference translations.
- BLEU accounts for brevity by applying a brevity penalty to shorter translations.
BLEU = BP * exp(∑ pn)

  *   BP (Brevity Penalty) adjusts the score for shorter.translations compared to reference translations. It's calculated as min(1, (reference_length / translated_length)).
  *   pn represents the precision of n-grams, calculated as the number of overlapping n-grams divided by the total number of n-grams in the translation.


- The score ranges from 0 to 1, with higher values indicating better translation quality.

#### ROUGE Score:

- ROUGE is commonly employed in text summarization tasks to evaluate the quality of machine-generated summaries against human-generated reference summaries.
- It measures the recall of n-grams in the machine-generated summary by comparing them to the reference summaries.
- ROUGE offers various variants such as ROUGE-N, ROUGE-L, and ROUGE-S, each focusing on different aspects of summarization quality.

  The formula for ROUGE score is as follows:
ROUGE = ∑ (Recall of n-grams)

  Where:

  *  Recall of n-grams is the number of n-grams that appear in both the machine-generated summary and the reference summaries divided by the total number of n-grams in the reference summaries.

- Like BLEU, the score ranges from 0 to 1, with higher values indicating better summary quality.


In [ ]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

   return result

In [ ]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 8
PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="/content/drive/MyDrive/forseAI/Model/Model1.0",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.182300,0.227213,0.953785,0.907361,0.953785,0.953890
2,0.154800,0.125345,0.948243,0.896069,0.948139,0.948243
3,0.135800,0.115165,0.953785,0.907361,0.953785,0.953890


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=7173, training_loss=0.18225629363252116, metrics={'train_runtime': 3446.5199, 'train_samples_per_second': 16.646, 'train_steps_per_second': 2.081, 'total_flos': 9462437167472640.0, 'train_loss': 0.18225629363252116, 'epoch': 3.0})

#Inference

## Online prediction

## INFERENCE  

Space - https://huggingface.co/spaces/Jyotiyadav/InsuranceClaim

Model - https://huggingface.co/Jyotiyadav/InsuranceModel1.0


In [ ]:
last_checkpoint = "Jyotiyadav/InsuranceModel1.0"

finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
